In [1]:
%load_ext autoreload
%autoreload 2
%cd /home/ftei-dsw/projects/weather-gan

/home/ftei-dsw/projects/weather-gan


In [19]:
import torch
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

from src.model import Generator, Discriminator
from src.trainer import WeatherGenerator
from src.data import DataModule
from src.utils import get_version, show_config
from src.configuration import Config
from src.trainer import training

# start training

In [21]:
show_config(Config())


################ configuration ################
-------------------- paths --------------------
tensorbard: /home/ftei-dsw/data/weather-gan/tensorboard/
checkpoint: /home/ftei-dsw/data/weather-gan/checkpoints/
config: /home/ftei-dsw/data/weather-gan/config-files/
results: /home/ftei-dsw/data/weather-gan/results/
input: /home/ftei-dsw/data/weather-gan/datasets/monthly_gfdl_historical.nc
target: /home/ftei-dsw/data/weather-gan/datasets/daily_gfdl_historical.nc
-------------------- splits  -------------------
training:   1970 - 2000
validation: 2001 - 2014
test:       2000 - 2014
------------------ hyperparams -----------------
epochs: 210
train batch size: 32
test batch size: 64
lr: 0.0002
latent dim: 8
generator channels: 64
generator num. residual blocks: 5
generator num. downsampling: 2
apply dropout: False
discriminator channels: 32
discriminator num layers: 3
num discriminator steps: 5


In [22]:
training(Config(), enable_profiler=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/opt/conda/envs/singularity/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 6.7 M 
1 | discriminator | Discriminator | 693 K 
------------------------------------------------
7.4 M     Trainable params
0         Non-trainable params
7.4 M     Total params
29.480    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/opt/conda/envs/singularity/lib/python3.9/site-packages/pytorch_lightning/loops/optimization/closure.py:35: LightningDeprecationWarning: One of the returned values {'progress_bar', 'log'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  rank_zero_deprecation(
/opt/conda/envs/singularity/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


# Debug